 ### 1. 서울시 공공데이터 제공 장소 116곳 리스트 뽑기


In [14]:
##### API 가져와보기
import requests
import pandas as pd 
import json
from urllib.parse import quote
from bs4 import BeautifulSoup
import re

# 서울 지역 116곳 번호 뽑기
area_list = []

for i in range(1,117):
    area = f"POI{i:03d}"

    # API 키
    api_key = '6e51654f776b353335387168745664'
    # API URL 구성
    url = f'http://openapi.seoul.go.kr:8088/{api_key}/xml/citydata/1/1000/{area}'
    # API 요청, 응답
    response = requests.get(url)
    xml_data = response.text
    # xml파싱
    soup = BeautifulSoup(xml_data, 'lxml-xml')
    area_name = soup.find('AREA_NM').text
    time =int(re.sub(r'[^\w]|[_]', '', soup.find('PPLTN_TIME').text))
    print(area_name, time)







강남 MICE 관광특구 202503270000
동대문 관광특구 202503270000
명동 관광특구 202503270000
이태원 관광특구 202503270000
잠실 관광특구 202503270000
종로·청계 관광특구 202503270000
홍대 관광특구 202503270000
경복궁 202503270000
광화문·덕수궁 202503270000
보신각 202503270000
서울 암사동 유적 202503270000
창덕궁·종묘 202503270000
가산디지털단지역 202503270000
강남역 202503270000
건대입구역 202503270000
고덕역 202503270000
고속터미널역 202503270000
교대역 202503270000
구로디지털단지역 202503270000
구로역 202503270000
군자역 202503270000
남구로역 202503270000
대림역 202503270000
동대문역 202503270000
뚝섬역 202503270000
미아사거리역 202503270000
발산역 202503270000
북한산우이역 202503270000
사당역 202503270000
삼각지역 202503270000
서울대입구역 202503270000
서울식물원·마곡나루역 202503270000
서울역 202503270000
선릉역 202503270000
성신여대입구역 202503270000
수유역 202503270000
신논현역·논현역 202503270000
신도림역 202503270000
신림역 202503270000
신촌·이대역 202503270000
양재역 202503270000
역삼역 202503270000
연신내역 202503270000
오목교역·목동운동장 202503270000
왕십리역 202503270000
용산역 202503270000
이태원역 202503270000
장지역 202503270000
장한평역 202503270000
천호역 202503270000
총신대입구(이수)역 202503270000
충정로역 20250327000

### 2. 가져온 지역 한개에서 요소 가져오기 

In [404]:
# 특정 요소 내의 모든 항목 찾기
live_ppltn = soup.find('LIVE_PPLTN_STTS')
#지역
area_nm = live_ppltn.find('AREA_NM').text
#혼잡도
congestion = live_ppltn.find('AREA_CONGEST_LVL').text
print(f"지역: {area_nm}, 혼잡도: {congestion}")
#자치구 
address = soup.find('ADDRESS').text
print(address)


지역: 낙산공원·이화마을, 혼잡도: 여유


AttributeError: 'NoneType' object has no attribute 'text'

#### 2.1데이터분석에 필요한 값들이 잘나오는지 하나씩 확인해보자

In [ ]:

## 서울시 실시간 데이터 가져오기

#데이트 날씨 점수 = 강수점수(30) + 기온점수(20) + 대기질점수(15) + 하늘상태점수(15) 

#### 가져올 값
#핫스팟 장소명(LIVE_PPLTN_STTS), 

#### 날씨 정보
#기온(TEMP), 강수(PRECIPITATION),강수확률(RAIN_CHANCE), 강수형태(PRECPT_TYPE), 대기질(AIR_IDX), 하늘상태(SKY_STTS), 습도(HUMIDITY), 자외선(UV_INDEX)

#### 혼잡도 정보
#장소 혼잡도 지표(AREA_CONGEST_LVL), 실시간 상권현황 (LIVE_CMRCL_STTS),  장소 예측 혼잡도 지표 (FCST_CONGEST_LVL), 장소 실시간 상권 현황(AREA_CMRCL_LVL)

##불쾌지수 = 1.8 x 기온 – 0.55 x (1 – 습도) x (1.8 x 기온 – 26) + 32

In [406]:
rain_mm =[]
# 강수량 (-일때 0으로 변경)
rain_mm_list = soup.find('PRECIPITATION')
for rain in rain_mm_list:
    rain = rain.text
    if rain == "-":
        rain_mm.append("0")
    else:
        rain_mm.append(rain)
    print(rain)
    
print(len(rain_mm))

-
1


In [ ]:
#온도
temp = soup.find_all('TEMP')
print(temp)
print(len(temp))

[<TEMP>15.2</TEMP>, <TEMP>14</TEMP>, <TEMP>13</TEMP>, <TEMP>12</TEMP>, <TEMP>10</TEMP>, <TEMP>10</TEMP>, <TEMP>9</TEMP>, <TEMP>9</TEMP>, <TEMP>9</TEMP>, <TEMP>9</TEMP>, <TEMP>9</TEMP>, <TEMP>8</TEMP>, <TEMP>8</TEMP>, <TEMP>8</TEMP>, <TEMP>9</TEMP>, <TEMP>10</TEMP>, <TEMP>12</TEMP>, <TEMP>14</TEMP>, <TEMP>17</TEMP>, <TEMP>19</TEMP>, <TEMP>20</TEMP>, <TEMP>17</TEMP>, <TEMP>16</TEMP>, <TEMP>14</TEMP>, <TEMP>13</TEMP>]
25


In [ ]:
#강수확률
rain_pct = soup.find_all('RAIN_CHANCE') 
print(rain_pct)
print(len(rain_pct))

[<RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>30</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>0</RAIN_CHANCE>, <RAIN_CHANCE>20</RAIN_CHANCE>]
24


In [ ]:
#강수형태
rain_type = soup.find_all('PRECPT_TYPE') # 없음 출력
print(rain_type)
print(len(rain_type))

[<PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>, <PRECPT_TYPE>없음</PRECPT_TYPE>]
25


In [ ]:
#대기질
air_idx = soup.find_all('AIR_IDX') #24동안 1하나만
print(air_idx)
print(len(air_idx))

[<AIR_IDX>나쁨</AIR_IDX>]
1


In [ ]:
#하늘상태
sty_stts = soup.find_all('SKY_STTS') #24동안 1하나만
print(sty_stts)
print(len(sty_stts))

[<SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>흐림</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>맑음</SKY_STTS>, <SKY_STTS>구름많음</SKY_STTS>]
24


In [ ]:
#습도
humidity = soup.find_all('HUMIDITY') 
print(humidity)
print(len(humidity))

[<HUMIDITY>64</HUMIDITY>]
1


#### 오후 온도 구하기

In [ ]:
from collections import Counter

fcst24hours = soup.find('FCST24HOURS')  # 단일 컨테이너 요소를 찾음
fcst_items = fcst24hours.find_all('FCST_DT')  # 또는 실제 항목 태그명으로 변경
temp_list=[]
sky_status_list = []
for item in fcst_items:
    hours = item.text
   
    # 원하는 시간대 확인 (12시~18시)
    if any(time in hours for time in ['1200', '1300', '1400', '1500', '1600', '1700', '1800']):

        
        parent = item.parent
        temp = int(parent.find('TEMP').text)
        temp_list.append(temp)
        sky_status_list.append(parent.find('SKY_STTS').text) #24동안 1하나만
        print(sty_stts)
# 오후 하늘상태 구하기
counter = Counter(sky_status_list)

# 온도 평균 구하기(합/개수)
avg_temp = int(sum(temp_list) / len(temp_list))
print(avg_temp)

구름많음
구름많음
구름많음
흐림
구름많음
맑음
맑음
16


In [ ]:
# 반복해서 구해야할 온도계수


### 오후날씨 함수만들기

In [ ]:
#리스트 가져오기
def get_after_wether(value):
    from collections import Counter

    fcst24hours = soup.find('FCST24HOURS')  # 단일 컨테이너 요소를 찾음
    fcst_items = fcst24hours.find_all('FCST_DT')  # 또는 실제 항목 태그명으로 변경

    after_list =[]
    #리스트마다 행하나씩 뽑기
    for item in fcst_items:
        hours = item.text
        
        if any(time in hours for time in ['1100','1200', '1300', '1400', '1500', '1600', '1700', '1800']):
            #상위 요소로 올라가기
            parent = item.parent
            
            #value가 온도일때 온도값 주기
            if value == 'temp':
                try:
                    temp = int(parent.find('TEMP').text)
                    after_list.append(temp)
                except:
                    print("온도값에 오류가 발생")
            #하늘상태일때 하늘상태값 주기 # 맑음 문자로 나옴 카운드 ?
            elif value == 'sky_stts':
                try:
                    sky_stts = parent.find('SKY_STTS').text
                    after_list.append(sky_stts)
                    print(sky_stts)
                except:
                    print("하늘상태값에 오류가 발생")
            #강수확률일때 강수확률값주기
            elif value == 'rain_chance':
                try:
                    rain_chance = int(parent.find('RAIN_CHANCE').text)
                    
                    after_list.append(rain_chance)
                    print(rain_chance)
                except:
                    print("강수확률값에 오류가 발생")

            # 강수량일때 (-일때 0으로 변경)
            elif value == 'rain_mm':
                try:
                    rain = parent.find('PRECIPITATION').text
                    if rain == "-":
                        after_list.append(int("0"))
                    else:
                        after_list.append(int(rain))
                    print(rain)
                except:
                    #오류발생
                    print("강수량 값에 대한 오류가 발생")   

    # 리스트 평균 구하기(합/개수)
    if value in ['temp', 'rain_chance', 'rain_mm']:
        avg_after = int(sum(after_list) / len(after_list))
        return avg_after
    else:
        return Counter(after_list).most_common(1)[0][0]

           

### 인구혼잡도 함수

In [ ]:

        
def get_many_people():
    from collections import Counter
    fcst24hours = soup.find('FCST_PPLTN')  # 단일 컨테이너 요소를 찾음
    fcst_items = fcst24hours.find_all('FCST_TIME')  # 또는 실제 항목 태그명으로 변경
    after_list =[]
    #리스트마다 행하나씩 뽑기
    for item in fcst_items:
        print(item)
        hours = item.text
        print(hours)
        if any(time in hours for time in ['11:00','12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00']):
            try:
                parent= item.parent
                many_level=parent.find('FCST_CONGEST_LVL').text
                after_list.append(many_level)
                print(many_level)
            except:
                print("혼잡도값에 대한 오류")
    
    return Counter(after_list).most_common(1)[0][0]


In [ ]:
aaa = get_after_wether('sky_stts')
print("값 :"+ aaa)

흐림
구름많음
구름많음
흐림
구름많음
맑음
맑음
구름많음
값 :구름많음


In [ ]:
bbb = get_many_people()
print(bbb)

IndexError: list index out of range

In [ ]:
#자외선
uv_index = soup.find_all('UV_INDEX') 
print(uv_index)
print(len(uv_index))

[<UV_INDEX>낮음</UV_INDEX>]
1


#### 2.2 실시간 인구 지표값 가져오기

In [ ]:
area_name = soup.find_all('AREA_NM') 
parent = area_name.parent
area_person_many = soup.find_all('AREA_CONGEST_LVL') 
live_cmrcl_stts = soup.find_all('LIVE_CMRCL_STTS') 
fcst_person_many = soup.find_all('FCST_CONGEST_LVL') # 12시간 예측
area_cmrcl_lvl = soup.find_all('AREA_CMRCL_LVL') 

fcst_person_many = soup.find_all('FCST_CONGEST_LVL') # 12시간 예측



#핫스팟장소명
print(area_name)
print(len(area_name))
# 장소 혼잡도 지표
print(area_person_many)
print(len(area_person_many))
# 실시간 상권현황
print(live_cmrcl_stts)
print(len(live_cmrcl_stts))
# 장소 예측 혼잡도 지표 
print(fcst_person_many)
print(len(fcst_person_many))
# 장소 실시간 상권 현황
print(area_cmrcl_lvl)
print(len(area_cmrcl_lvl))

[<AREA_NM>노들섬</AREA_NM>, <AREA_NM>노들섬</AREA_NM>]
2
[<AREA_CONGEST_LVL>여유</AREA_CONGEST_LVL>]
1
[<LIVE_CMRCL_STTS/>]
1
[<FCST_CONGEST_LVL>보통</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>, <FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL>]
12
[]
0


In [ ]:
fcst_person_many = soup.find_all('FCST_TIME') # 12시간 예측

for i in fcst_person_many:
    parent = i.parent
    print(parent)
    print(i)


print(len(fcst_person_many))

<FCST_PPLTN><FCST_TIME>2025-03-25 00:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>7500</FCST_PPLTN_MIN><FCST_PPLTN_MAX>8000</FCST_PPLTN_MAX></FCST_PPLTN>
<FCST_TIME>2025-03-25 00:00</FCST_TIME>
<FCST_PPLTN><FCST_TIME>2025-03-25 01:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>6000</FCST_PPLTN_MIN><FCST_PPLTN_MAX>6500</FCST_PPLTN_MAX></FCST_PPLTN>
<FCST_TIME>2025-03-25 01:00</FCST_TIME>
<FCST_PPLTN><FCST_TIME>2025-03-25 02:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>5000</FCST_PPLTN_MIN><FCST_PPLTN_MAX>5500</FCST_PPLTN_MAX></FCST_PPLTN>
<FCST_TIME>2025-03-25 02:00</FCST_TIME>
<FCST_PPLTN><FCST_TIME>2025-03-25 03:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>4500</FCST_PPLTN_MIN><FCST_PPLTN_MAX>5000</FCST_PPLTN_MAX></FCST_PPLTN>
<FCST_TIME>2025-03-25 03:00</FCST_TIME>
<FCST_PPLTN><FCST_TIME>2025-03-25 04:00</FCST_TIME><FCST_CONGEST_LVL>여유</FCST_CONGEST_LVL><FCST_PPLTN_MIN>4500</FCST_PPLTN_MIN><

### 3.합쳐보기

### 함수

-----------

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
import re
from selenium.common.exceptions import TimeoutException

#이름 네이버 지도에 검색해서 첫번째 항목 주소 구(서울 아니면 시이름)가져오는 함수
def get_addr(not_addr_name):  
    # chrome드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
    options.add_argument('window-size=1380,900')
    options.add_argument("--headless")  # 👈 화면 없이 실행 (필수)
    driver = webdriver.Chrome(options=options)

    # 네이버 검색
    search_url = f"https://map.naver.com/p/search/{urllib.parse.quote(not_addr_name)}"
    driver.get(search_url)
    try:
        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CLASS_NAME, "spinner_wrap")))
    except TimeoutException:
        print("iframe 로딩 실패 - 새로고침 시도")
        driver.refresh()

        try:
            WebDriverWait(driver, 10).until(
                EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe"))
            )
        except TimeoutException:
            print("iframe 두 번째 로딩도 실패 - 함수 종료")
            return None
        

    #iframe 전환
    def switch_left():
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))
    switch_left()


    #뷰티플 소프로 검색한 첫번째 장소 주소 가져오기
    soup = BeautifulSoup(driver.page_source, "html.parser")
    search_list = soup.find(id="_pcmap_list_scroll_container").find_all('li')
    search_gu_name = search_list[0].find(class_="KgfA6 D7FgR").find(class_="Pb4bU").text
    split_gu_name = search_gu_name.split()


    # 가져온 값이 서울이면 구 이름 리터 아니면 시이름 리턴
    if '서울' in split_gu_name:
        gu_name = split_gu_name[1]
    else :
        gu_name = "경기도"

    driver.quit()

    return gu_name


In [23]:
import mysql.connector

def execute_sql(query, values=None):
    ojk = mysql.connector.connect(
        host = "database-1.cnu82kme6p4d.ap-northeast-2.rds.amazonaws.com",
        port = 3306,
        user = "eda",
        password = "ojk0707",
        database = "ojk"
    )
    cursor = ojk.cursor()
    cursor.execute(query, values)
    ojk.commit()
    cursor.close()

------------

## DB에 저장하기

In [24]:
##### API 가져와보기
import requests
import pandas as pd 
from urllib.parse import quote
from bs4 import BeautifulSoup
import re
import mysql.connector


# 서울 지역 116곳 번호 뽑기
total_listimet = []

for i in range(1,117):
    area_list = []
    #지역 코드 값 자동 완성
    area = f"POI{i:03d}"
    # API 키
    api_key = '6e51654f776b353335387168745664'
    # API URL 구성
    url = f'http://openapi.seoul.go.kr:8088/{api_key}/xml/citydata/1/2/{area}'
    # API 요청, 응답
    response = requests.get(url)
    xml_data = response.text
    # xml파싱
    soup = BeautifulSoup(xml_data, 'lxml-xml')

    # 필요한 값들 가져오기
    # datetime
    update_time =int(re.sub(r'[^\w]|[_]', '', soup.find('PPLTN_TIME').text))
    #명소명
    area_nm = soup.find('AREA_NM').text
    #최대 인구
    max_pepl = int(soup.find('AREA_PPLTN_MAX').text)
    #최소 인구
    min_pepl = int(soup.find('AREA_PPLTN_MIN').text)
    #기온
    temp = float(soup.find('TEMP').text)
    #강수
    rain_mm = soup.find('PRECIPITATION').text
    if rain_mm == "-":
        rain_mm = 0
    else:
        rain_mm = int(rain_mm)
    #강수확률
    rain_per = int(soup.find('RAIN_CHANCE').text)
    #강수형태
    rain_type = soup.find('PRECPT_TYPE').text
    #대기질
    air_type = soup.find('AIR_IDX').text
    #하늘상태
    sky_stts = soup.find('SKY_STTS').text
    #습도
    air_water = soup.find('HUMIDITY').text
    #풍향
    wind_level = float(soup.find('WIND_SPD').text)
    #구
    try:
        address = soup.find('ADDRESS').text
        gu = re.search(r'[가-힣]+구', address).group()
    #주소가 없을때 네이버 검색
    except:
        not_addr_name = area_nm.split("·")[0]
        gu= get_addr(not_addr_name)
    area_list += [update_time,area_nm, gu, min_pepl, max_pepl, temp, rain_mm, rain_per, rain_type, air_type, sky_stts, air_water, wind_level]
    
    query = """INSERT INTO spot (datetime, name, gu, min_population, max_population, temp, precipitation,
        rain_chance, precpt_type, air_idx, sky_stts, humidity, wind_spd) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
    execute_sql(query, area_list)
  
    
    print(area_list)

[202503270015, '강남 MICE 관광특구', '강남구', 1000, 1500, 17.0, 0, 60, '없음', '나쁨', '구름많음', '37', 0.6]
[202503270015, '동대문 관광특구', '중구', 18000, 20000, 18.5, 0, 60, '없음', '나쁨', '구름많음', '42', 2.6]
[202503270015, '명동 관광특구', '중구', 7000, 7500, 18.5, 0, 60, '없음', '나쁨', '구름많음', '42', 2.6]
[202503270015, '이태원 관광특구', '용산구', 4500, 5000, 18.3, 0, 60, '없음', '나쁨', '구름많음', '36', 1.1]
[202503270015, '잠실 관광특구', '송파구', 16000, 18000, 18.0, 0, 60, '없음', '나쁨', '구름많음', '35', 0.6]
[202503270015, '종로·청계 관광특구', '종로구', 8000, 8500, 18.5, 0, 60, '없음', '나쁨', '구름많음', '42', 2.6]
[202503270015, '홍대 관광특구', '마포구', 26000, 28000, 16.7, 0, 60, '없음', '나쁨', '구름많음', '45', 0.5]
[202503270015, '경복궁', '종로구', 300, 400, 18.5, 0, 60, '없음', '나쁨', '구름많음', '42', 2.6]
[202503270015, '광화문·덕수궁', '중구', 4000, 4500, 18.5, 0, 60, '없음', '나쁨', '구름많음', '42', 2.6]
[202503270015, '보신각', '종로구', 2000, 2500, 18.5, 0, 60, '없음', '나쁨', '구름많음', '42', 2.6]
[202503270015, '서울 암사동 유적', '강동구', 300, 400, 18.0, 0, 60, '없음', '나쁨', '구름많음', '35', 0.6]
[202503270015, '창덕